In [4]:
import torch

# 创建维度为[256, 1]的张量
tensor1 = torch.randn(256, 1,1)

# 创建维度为[8, 256, 16, 16]的张量
tensor2 = torch.randn(8, 256, 16, 16)

# 对tensor2进行转置操作，使其满足矩阵乘法的维度匹配规则
# tensor2_transposed = tensor2.permute(0, 2, 3, 1).view(8, -1, 256)

# 进行矩阵乘法
result = tensor1*tensor2



print(result.shape)  

torch.Size([8, 256, 16, 16])


In [ ]:


class CMUNeXtBlock_MK(nn.Module):  # 根据SCSA灵感，直接在一个block中为不同的channel设置不同的卷积核，同时引入通道注意力
    def __init__(self, ch_in, ch_out, depth=1, k=3): # ch_in 需要是4的倍数
        super(CMUNeXtBlock, self).__init__()
        self.block = nn.Sequential(
            *[nn.Sequential(
                Residual(nn.Sequential(
                    # deep wise
                    nn.Conv2d(ch_in, ch_in, kernel_size=(k, k), groups=ch_in, padding=(k // 2, k // 2)),
                    nn.GELU(),
                    nn.BatchNorm2d(ch_in)
                )),
                nn.Conv2d(ch_in, ch_in * 4, kernel_size=(1, 1)),
                nn.GELU(),
                nn.BatchNorm2d(ch_in * 4),
                nn.Conv2d(ch_in * 4, ch_in, kernel_size=(1, 1)),
                nn.GELU(),
                nn.BatchNorm2d(ch_in)
            ) for i in range(depth)]
        )
        self.up = conv_block(ch_in, ch_out)
        self.group_chans = group_chans = self.dim // 4


    def forward(self, x):
        b, c, h_, w_ = x.size()
        l_x_h, g_x_h_s, g_x_h_m, g_x_h_l = torch.split(x, self.group_chans, dim=1)
        x = self.block(x)
        x = self.up(x)
        return x
